# 05 Quantile forecasting using Gradient Boosted Trees

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
QUANTILE_LB, QUANTILE_UB = 0.1, 0.9

stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Loading stock data

In [5]:
stock_data = su.StockData(STOCK_SYMBOL)
stock_data

Symbol: HDFCBANK
Total records: 1049
First record: 2020-01-01
Last record: 2024-02-26

## Modelling

### Target columns

In [6]:
target_cols = stock_data.standardized.filter(regex = "Target.*").columns.to_list()
target_cols

['Target 3D', 'Target 7D', 'Target 15D', 'Target 30D']

In [7]:
pred_input_df = stock_data.standardized.drop(columns = target_cols)
pred_input_df.shape

(1049, 42)

In [8]:
stock_data.standardized[target_cols].describe()

,Target 3D,Target 7D,Target 15D,Target 30D
count,1046.000000,1042.000000,1034.000000,1019.000000
mean,1.000772,1.001874,1.003761,1.008427
std,0.030943,0.046347,0.065198,0.088580
min,0.820000,0.718000,0.668000,0.629000
25%,0.985000,0.977000,0.967000,0.959500
50%,1.000000,1.002000,1.001500,1.006000
75%,1.016750,1.025000,1.042000,1.058000
max,1.178000,1.219000,1.231000,1.322000


### Data processing

In [9]:
def get_training_data(target_col: str):
    print(f"Target: {target_col}")
    X_df = stock_data.standardized[
        stock_data.standardized[target_col].notnull()
    ].drop(columns = target_cols).copy()
    y = stock_data.standardized[
        stock_data.standardized[target_col].notnull()
    ][target_col].copy()
    print(f"X.shape: {X_df.shape}")
    print(f"y.shape: {y.shape}")

    return X_df, y

### Grid search parameters

In [10]:
param_dict = {
    "n_estimators": [100, 125, 150],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [2, 3, 4],
    "max_features": ["log2", "sqrt", 0.25],
    "subsample": [0.75, 1.0]
}

feature_importances = pd.DataFrame(
    index = stock_data.standardized.drop(columns = target_cols).columns.to_list()
)

### Model building

In [11]:
def get_model():
    gb_model = GradientBoostingRegressor(
        loss = "squared_error",
        random_state = cnst.RANDOM_STATE
    )

    grid_cv = RandomizedSearchCV(
        gb_model,
        param_dict,
        n_iter = 12,
        cv = 5,
        n_jobs = -1,
        random_state = cnst.RANDOM_STATE
    )

    return grid_cv

def get_quantile_model(quantile: float):
    gb_model = GradientBoostingRegressor(
        loss = "quantile",
        alpha = quantile,
        random_state = cnst.RANDOM_STATE
    )

    grid_cv = RandomizedSearchCV(
        gb_model,
        param_dict,
        n_iter = 12,
        cv = 5,
        n_jobs = -1,
        random_state = cnst.RANDOM_STATE
    )

    return grid_cv

def print_results(y, preds):
    print(f"Target std: {y.std():.3f}")
    print(f"R2: {r2_score(y, preds):.3f}")
    print(f"MSE: {mean_squared_error(y, preds, squared = False):.3f}")
    print(f"MAE: {mean_absolute_error(y, preds):.3f}")

## Training

### `Target 3D`

#### Forecasting model

In [12]:
target_col = target_cols[0]
X, y = get_training_data(target_col)

Target: Target 3D
X.shape: (1046, 42)
y.shape: (1046,)


In [13]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 0.75,
 'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 3,
 'learning_rate': 0.05}

In [14]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.300187,0.011134,0.005264,0.001570,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.066658,-0.089246,-0.072345,-0.049385,-0.211841,-0.097895,0.058379,1
2,0.433380,0.019306,0.010666,0.011152,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.215254,-0.125270,-0.074321,-0.036284,-0.173489,-0.124924,0.064709,2
11,0.373488,0.040345,0.002765,0.000757,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.178605,-0.192538,-0.079846,-0.017111,-0.258094,-0.145239,0.085767,3
4,0.398225,0.083666,0.004259,0.001095,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.170025,-0.222955,-0.087386,-0.045034,-0.361074,-0.177295,0.110896,4
8,0.483007,0.055795,0.006526,0.001720,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.246495,-0.174673,-0.132481,-0.055515,-0.283773,-0.178587,0.081195,5


In [15]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.031
R2: 0.416
MSE: 0.024
MAE: 0.018


#### Lower quantile model

In [16]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 0.75,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [17]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [18]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 3D,Pred Target 3D,LB Target 3D,UB Target 3D
1036,2024-02-08,1403.05,1394.45,1406.79,1386.69,1431.00
1037,2024-02-09,1403.60,1384.05,1406.12,1369.76,1438.70
1038,2024-02-12,1390.00,1414.05,1402.47,1364.82,1418.20
1039,2024-02-13,1394.45,1419.90,1400.26,1353.92,1427.82
1040,2024-02-14,1384.05,1417.10,1389.03,1346.79,1417.57
1041,2024-02-15,1414.05,1454.30,1421.22,1376.77,1453.55
1042,2024-02-16,1419.90,1439.15,1421.20,1390.50,1448.21
1043,2024-02-19,1417.10,1419.55,1417.26,1376.43,1432.34
1044,2024-02-20,1454.30,1420.60,1446.31,1399.86,1483.12
1045,2024-02-21,1439.15,1422.30,1431.12,1391.73,1474.87


### `Target 7D`

#### Forecasting model

In [19]:
target_col = target_cols[1]
X, y = get_training_data(target_col)

Target: Target 7D
X.shape: (1042, 42)
y.shape: (1042,)


In [20]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 0.75,
 'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 3,
 'learning_rate': 0.05}

In [21]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.222316,0.012415,0.003044,0.000592,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.212472,-0.409199,-0.131862,-0.189185,-0.229286,-0.234401,0.093402,1
2,0.412292,0.020967,0.004852,0.001329,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.243395,-0.496641,-0.217033,-0.174433,-0.179581,-0.262216,0.119904,2
11,0.313061,0.009383,0.002628,0.000567,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.219641,-0.572133,-0.214868,-0.182955,-0.283585,-0.294637,0.142539,3
4,0.355606,0.072589,0.005454,0.002527,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.229695,-0.487477,-0.287153,-0.151993,-0.429649,-0.317193,0.124493,4
8,0.422041,0.012387,0.004636,0.001304,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.296438,-0.503339,-0.395144,-0.164829,-0.318764,-0.335703,0.111948,5


In [22]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.046
R2: 0.535
MSE: 0.032
MAE: 0.024


#### Lower quantile model

In [23]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 0.25,
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [24]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [25]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 7D,Pred Target 7D,LB Target 7D,UB Target 7D
1032,2024-02-02,1446.15,1394.45,1417.37,1392.29,1475.25
1033,2024-02-05,1444.85,1384.05,1420.15,1384.33,1466.99
1034,2024-02-06,1444.10,1414.05,1425.03,1401.34,1477.55
1035,2024-02-07,1429.95,1419.90,1421.70,1402.45,1460.60
1036,2024-02-08,1403.05,1417.10,1430.34,1404.29,1445.96
1037,2024-02-09,1403.60,1454.30,1428.60,1400.97,1453.60
1038,2024-02-12,1390.00,1439.15,1413.38,1382.68,1438.68
1039,2024-02-13,1394.45,1419.55,1408.32,1391.97,1435.60
1040,2024-02-14,1384.05,1420.60,1409.69,1381.26,1433.38
1041,2024-02-15,1414.05,1422.30,1413.96,1374.74,1434.97


### `Target 15D`

#### Forecasting model

In [26]:
target_col = target_cols[2]
X, y = get_training_data(target_col)

Target: Target 15D
X.shape: (1034, 42)
y.shape: (1034,)


In [27]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 1.0,
 'n_estimators': 100,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.05}

In [28]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.514688,0.117344,0.008527,0.004084,1.0,100,log2,4,0.05,"{'subsample': 1.0, 'n_estimators': 100, 'max_f...",-0.040553,-0.517736,-0.489686,-0.635865,-0.408969,-0.418562,0.202533,1
10,0.302638,0.011499,0.005690,0.001614,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.049762,-0.683729,-0.388461,-0.520941,-0.507410,-0.430060,0.212107,2
4,0.400716,0.078132,0.004864,0.001191,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.003325,-0.489420,-0.485693,-0.562894,-0.719374,-0.452141,0.239846,3
8,0.514499,0.044960,0.005455,0.000974,1.0,125,sqrt,4,0.05,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.113624,-0.932946,-0.613601,-0.547511,-0.606154,-0.562767,0.262092,4
11,0.424085,0.026297,0.003786,0.000670,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.138337,-0.665488,-0.698906,-0.660652,-0.682082,-0.569093,0.215798,5


In [29]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.065
R2: 0.785
MSE: 0.030
MAE: 0.024


#### Lower quantile model

In [30]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 0.25,
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [31]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [32]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 15D,Pred Target 15D,LB Target 15D,UB Target 15D
1024,2024-01-20,1478.85,1394.45,1420.90,1394.60,1485.80
1025,2024-01-23,1427.35,1384.05,1378.68,1367.22,1445.51
1026,2024-01-24,1455.90,1414.05,1438.55,1413.41,1499.15
1027,2024-01-25,1434.90,1419.90,1430.23,1389.84,1477.59
1028,2024-01-29,1454.65,1417.10,1434.23,1411.99,1507.52
1029,2024-01-30,1444.30,1454.30,1439.49,1397.18,1498.93
1030,2024-01-31,1462.55,1439.15,1447.78,1413.48,1515.63
1031,2024-02-01,1466.35,1419.55,1431.66,1410.51,1499.16
1032,2024-02-02,1446.15,1420.60,1424.46,1395.02,1502.15
1033,2024-02-05,1444.85,1422.30,1428.91,1400.04,1489.75


### `Target 30D`

#### Forecasting model

In [33]:
target_col = target_cols[3]
X, y = get_training_data(target_col)

Target: Target 30D
X.shape: (1019, 42)
y.shape: (1019,)


In [34]:
model = get_model()
model.fit(X, y)
model.best_params_

{'subsample': 0.75,
 'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 3,
 'learning_rate': 0.05}

In [35]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score').iloc[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_max_features,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.343650,0.043459,0.007564,0.004811,0.75,100,sqrt,3,0.05,"{'subsample': 0.75, 'n_estimators': 100, 'max_...",-0.147687,-0.331994,-1.052969,-0.402030,-0.218698,-0.430675,0.323365,1
11,0.466605,0.028289,0.003469,0.000519,0.75,150,sqrt,4,0.05,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.045618,-0.590148,-1.419630,-0.422383,-0.231287,-0.541813,0.475352,2
4,0.374546,0.031858,0.014801,0.009144,1.0,125,sqrt,2,0.1,"{'subsample': 1.0, 'n_estimators': 125, 'max_f...",-0.363835,-0.448360,-1.314999,-0.593042,-0.066286,-0.557304,0.416119,3
5,1.050504,0.071448,0.006643,0.001858,1.0,150,0.25,4,0.1,"{'subsample': 1.0, 'n_estimators': 150, 'max_f...",-0.203736,0.018150,-1.293859,-0.921488,-0.389324,-0.558051,0.481563,4
0,0.447793,0.034637,0.005399,0.000660,0.75,150,log2,4,0.1,"{'subsample': 0.75, 'n_estimators': 150, 'max_...",-0.254618,-0.379297,-1.347617,-0.679496,-0.314468,-0.595099,0.403648,5


In [36]:
feature_importances = feature_importances.join(
    pd.DataFrame(
        model.best_estimator_.feature_importances_,
        index = model.best_estimator_.feature_names_in_,
        columns = [target_col]
    )
)

preds = model.predict(X)
print_results(y, preds)

Target std: 0.089
R2: 0.785
MSE: 0.041
MAE: 0.033


#### Lower quantile model

In [37]:
lb_model = get_quantile_model(QUANTILE_LB)
lb_model.fit(X, y)
lb_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 0.25,
 'max_depth': 4,
 'learning_rate': 0.1}

#### Upper quantile model

In [38]:
ub_model = get_quantile_model(QUANTILE_UB)
ub_model.fit(X, y)
ub_model.best_params_

{'subsample': 1.0,
 'n_estimators': 150,
 'max_features': 'log2',
 'max_depth': 4,
 'learning_rate': 0.1}

#### Predictions

In [39]:
pred_col_name = f'Pred {target_col}'
lb_col_name, ub_col_name = f'LB {target_col}', f'UB {target_col}'

stock_data.processed[pred_col_name] = (model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[lb_col_name] = (lb_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)
stock_data.processed[ub_col_name] = (ub_model.predict(pred_input_df) * stock_data.processed['Close']).round(2)

stock_data.processed[['Date', 'Close', target_col, pred_col_name, lb_col_name, ub_col_name]].dropna().iloc[-10:, :]

,Date,Close,Target 30D,Pred Target 30D,LB Target 30D,UB Target 30D
1009,2024-01-01,1698.10,1394.45,1475.43,1393.49,1699.47
1010,2024-01-02,1699.10,1384.05,1453.24,1384.44,1681.03
1011,2024-01-03,1672.90,1414.05,1444.08,1413.00,1669.78
1012,2024-01-04,1690.85,1419.90,1456.40,1419.43,1675.86
1013,2024-01-05,1682.20,1417.10,1472.17,1414.69,1688.96
1014,2024-01-08,1663.45,1454.30,1462.85,1437.67,1669.27
1015,2024-01-09,1650.50,1439.15,1492.43,1437.77,1671.01
1016,2024-01-10,1655.95,1419.55,1480.28,1419.59,1605.21
1017,2024-01-11,1649.00,1420.60,1476.44,1419.54,1681.58
1018,2024-01-12,1641.20,1422.30,1466.64,1422.79,1661.52


## Feature importances

In [40]:
feature_importances['Mean'] = feature_importances.mean(axis = 1)
(feature_importances.sort_values('Mean', ascending = False) * 100).round(1)

,Target 3D,Target 7D,Target 15D,Target 30D,Mean
52W H,12.1,9.3,9.5,11.1,10.5
DayOfYear,4.5,6.6,7.8,12.7,7.9
PE,4.0,6.7,7.2,12.9,7.7
Range 60MA,4.6,5.9,6.8,10.4,6.9
52W L,2.6,4.4,6.0,7.8,5.2
Month,1.6,3.3,7.9,6.1,4.7
Range 30MA,3.4,4.7,4.4,6.1,4.7
Range 15MA,3.2,5.3,3.3,3.4,3.8
Range 7MA,3.1,5.7,4.3,1.8,3.7
VWAP 60MA,4.7,2.6,3.9,1.3,3.1


## Forecasts

In [41]:
stock_data.processed.filter(regex = "(Date)|(Close$)|(Pred.*)").iloc[-10:, :]

,Date,Close,Pred Target 3D,Pred Target 7D,Pred Target 15D,Pred Target 30D
1039,2024-02-13,1394.45,1400.26,1408.32,1379.30,1404.73
1040,2024-02-14,1384.05,1389.03,1409.69,1382.06,1372.73
1041,2024-02-15,1414.05,1421.22,1413.96,1380.40,1401.77
1042,2024-02-16,1419.90,1421.20,1417.21,1387.38,1414.51
1043,2024-02-19,1417.10,1417.26,1416.77,1387.59,1412.27
1044,2024-02-20,1454.30,1446.31,1454.14,1413.40,1436.46
1045,2024-02-21,1439.15,1431.12,1436.82,1397.19,1436.71
1046,2024-02-22,1419.55,1417.19,1444.73,1380.89,1424.58
1047,2024-02-23,1420.60,1419.77,1423.49,1380.89,1416.20
1048,2024-02-26,1422.30,1424.50,1429.57,1380.39,1417.23


In [42]:
latest_preds = stock_data.processed.iloc[-1]
print(f"Date: {latest_preds['Date'].date()}")
print(f"Close: {latest_preds['Close']}")

for target_col in target_cols:
    print(f"{target_col}: {latest_preds[f'Pred {target_col}']}", end = " ")
    print(f"({latest_preds[f'LB {target_col}']} to {latest_preds[f'UB {target_col}']})")

Date: 2024-02-26
Close: 1422.3
Target 3D: 1424.5 (1377.21 to 1455.82)
Target 7D: 1429.57 (1361.54 to 1447.92)
Target 15D: 1380.39 (1338.06 to 1495.72)
Target 30D: 1417.23 (1313.17 to 1559.7)
